# H$_2$ prototyping - occupations and density difference

In [9]:
import pyscf
from pyscf import gto, scf, dft, ao2mo, fci
import numpy as np
import scipy.linalg as lin
import matplotlib.pyplot as plt

In [2]:
npoints = 20
R_begin=0.5
R_end=6.0
R = np.linspace(start=R_begin,stop=R_end,num=npoints)

In [3]:
def get_entropy(mf):
    f = mf.get_occ()/2
    #print(f)
    f = f[(f>0) & (f<1)]
    return -2*(f*np.log(f) + (1-f)*np.log(1-f)).sum()

In [4]:
def run_rks(mol_str,smear=False,tau=500,basis='cc-pVTZ'):
    mol = gto.M(atom=mol_str, basis=basis)
    mol.smearing = smear
    mol.tau = tau
    mf = scf.RKS(mol)
    mf.xc='PBE'
    mf.run()
    h1 = mf.mo_coeff.T.dot(mf.get_hcore()).dot(mf.mo_coeff)
    T = mf.mo_coeff.T.dot(mf.mol.intor_symmetric('int1e_kin')).dot(mf.mo_coeff)
    rdm1=mf.make_rdm1()
    Ekin = np.einsum('pq,qp', T, rdm1)
    #print("Ts: ", Ekin)
    d = np.zeros(h1.shape[0])
    s = np.zeros(h1.shape[0])
    for i in np.arange(h1.shape[0]):
        d[i] = h1[i,i]
        s[i] = rdm1[i,i]
    return s, d, mf, Ekin, mf.energy_tot()

In [5]:
def run_fci(mol_str,basis='cc-pVTZ'):
    mol = gto.M(atom=mol_str, basis=basis)
    mf = scf.RHF(mol)
    mf.run()
    norb=np.shape(mf.get_occ())[0]
    h1 = mf.mo_coeff.T.dot(mf.get_hcore()).dot(mf.mo_coeff)
    T = mf.mo_coeff.T.dot(mf.mol.intor_symmetric('int1e_kin')).dot(mf.mo_coeff)
    eri = ao2mo.kernel(mol, mf.mo_coeff)
    cisolver = fci.direct_spin1.FCI(mol)
    e, ci = cisolver.kernel(h1, eri, h1.shape[1], mol.nelec, ecore=mol.energy_nuc())
    rdm1=cisolver.make_rdm1(fcivec=ci,norb=norb,nelec=mol.nelec)
    Ekin = np.einsum('pq,qp', T, rdm1)
    #print("exact T: ", Ekin)
    d = np.zeros(h1.shape[0])
    s = np.zeros(h1.shape[0])
    for i in np.arange(h1.shape[0]):
        d[i] = h1[i,i]
        s[i] = rdm1[i,i]
    return s, d, mf, Ekin, e


In [6]:
smear = False
occupations = []
energies = []
calculations = []
Ekin_KS = []
ener_KS = []
for i in np.arange(npoints):
    mol_str = "H 0 0 0; H 0 0 "+str(R[i])
    s, d, mf, Ekin, e = run_rks(mol_str=mol_str, smear=smear)
    occupations.append(s)
    energies.append(s)
    calculations.append(mf)
    Ekin_KS.append(Ekin)
    ener_KS.append(e)

converged SCF energy = -1.09191079551157
converged SCF energy = -1.16527378622365
converged SCF energy = -1.12881999115196
converged SCF energy = -1.07981515146401
converged SCF energy = -1.03687938302148
converged SCF energy = -1.00311829781504
converged SCF energy = -0.977811141837898
converged SCF energy = -0.959377001467066
converged SCF energy = -0.946207974172045
converged SCF energy = -0.936932953427283
converged SCF energy = -0.930474273914102
converged SCF energy = -0.926015751454567
converged SCF energy = -0.92295675707269
converged SCF energy = -0.920868535816567
converged SCF energy = -0.919450609167802
converged SCF energy = -0.91840968704719
converged SCF energy = -0.917855981813743
converged SCF energy = -0.917434919558024
converged SCF energy = -0.917161789461337
converged SCF energy = -0.915862210281631


In [7]:
tau = 8000
smear = True
occupations = []
energies = []
calculations = []
Ekin_KS_tau = []
ener_KS_tau = []

for i in np.arange(npoints):
    mol_str = "H 0 0 0; H 0 0 "+str(R[i])
    s, d, mf, Ekin, e = run_rks(mol_str=mol_str,tau=tau,smear=smear)
    occupations.append(s)
    energies.append(s)
    calculations.append(mf)
    if smear:
        mTS=-0.3166808991e-5*tau*get_entropy(mf)
    else:
        mTS = 0.0
    print(mTS)
    Ekin_KS_tau.append(Ekin)
    ener_KS_tau.append(e+mTS)

converged SCF energy = -1.09187252171842
-4.199046611156378e-05
converged SCF energy = -1.16500230262247
-0.00030538826392991045
converged SCF energy = -1.12717801693933
-0.0019213927066776954
converged SCF energy = -1.0732824206292
-0.008126901316708958
converged SCF energy = -1.02117918845049
-0.021397303905716942
converged SCF energy = -0.978603763590401
-0.03808226984939941
converged SCF energy = -0.949749589038281
-0.052068380138300306
converged SCF energy = -0.933155107949998
-0.060993622221675495
converged SCF energy = -0.924617987091732
-0.06581870334031147
converged SCF energy = -0.92049079949532
-0.06819576609072452
converged SCF energy = -0.918545001581727
-0.06931127011060634
converged SCF energy = -0.917624877346564
-0.06982296492644667
converged SCF energy = -0.917180456341411
-0.07005533687852439
converged SCF energy = -0.916959742922924
-0.07016001425396978
converged SCF energy = -0.916846896793624
-0.07020670206983202
converged SCF energy = -0.916789304137159
-0.070227

In [8]:
occupations = []
energies = []
calculations = []
Ekin_FCI = []
ener = []
for i in np.arange(npoints):
    mol_str = "H 0 0 0; H 0 0 "+str(R[i])
    s, d, mf, Ekin, e = run_fci(mol_str=mol_str)
    occupations.append(s)
    energies.append(s)
    calculations.append(mf)
    Ekin_FCI.append(Ekin)
    ener.append(e)

converged SCF energy = -1.06330677086093
converged SCF energy = -1.13104898043224
converged SCF energy = -1.08707872740849
converged SCF energy = -1.02926261880808
converged SCF energy = -0.976525021657721
converged SCF energy = -0.932295135965931
converged SCF energy = -0.896263433537805
converged SCF energy = -0.867269689665
converged SCF energy = -0.84405629079119
converged SCF energy = -0.825489700644205
converged SCF energy = -0.810630566559902
converged SCF energy = -0.798722268706303
converged SCF energy = -0.789159342990459
converged SCF energy = -0.781455737572377
converged SCF energy = -0.775218402534373
converged SCF energy = -0.770129324203516
converged SCF energy = -0.765933657032088
converged SCF energy = -0.762430220584095
converged SCF energy = -0.759462607638793
converged SCF energy = -0.756910815518384


In [ ]:
def plot_world(T_KS,T_KS_tau,T_FCI):
    T_KS = np.array(T_KS)
    T_KS_tau = np.array(T_KS_tau)
    T_FCI = np.array(T_FCI)
    plt.plot(R,T_FCI,label='FCI')
    plt.plot(R,T_KS,label='KS')
    plt.plot(R,T_KS_tau,label='KS_tau')
    plt.plot(R,T_KS-T_KS_tau,label=r'$\Delta$',linestyle='--')
    plt.legend()
    plt.show()

In [ ]:
plot_world(Ekin_KS,Ekin_KS_tau,Ekin_FCI)

In [ ]:
plot_world(ener_KS,ener_KS_tau,ener)

# prototyping natural orbitals

In [ ]:
SVD = lin.svd

In [ ]:
U, s, Vh = SVD(a=rdm1)

In [ ]:
h1_natural = np.dot(Vh.T, np.dot(h1, U.T))

In [ ]:
d = np.zeros(h1_natural.shape[0])
s = np.zeros(h1_natural.shape[0])
for i in np.arange(h1_natural.shape[0]):
    d[i] = h1_natural[i,i]
    s[i] = rdm1[i,i]

In [ ]:
plt.scatter(x=d,y=s)
plt.show()